# Retail Sales Data Analysis

## Import Libraries and Establish Database Connection

In [ ]:
# Importing the pyodbc module for connecting to SQL Server
import pyodbc
# Importing pandas library for data manipulation
import pandas as pd

# Define the server name and database name
server = 'IN3549317W1'  # or your server name
database = 'RetailSales'  # your database name

# Establish a connection to the SQL Server using pyodbc
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=' + server + ';DATABASE=' + database + ';Trusted_Connection=yes;')
# Create a cursor object to execute SQL queries
cursor = cnxn.cursor()
# Print the cursor object (optional, for verification)
print(cursor)

## Importing the Retail Sales - Data

In [ ]:
# Read the Excel file "Retail_Brands.xlsx" into a pandas DataFrame
df_Retail_Brands = pd.read_excel("Retail_Brands.xlsx")

In [ ]:
# Display the first few rows of the DataFrame
df_Retail_Brands.head()

In [ ]:
# Read the Excel file "Retail_Employees.xlsx" into a pandas DataFrame
df_Retail_Employees = pd.read_excel("Retail_Employees.xlsx")

# Display information about the DataFrame (e.g., column names, data types)
df_Retail_Employees.info()

In [ ]:
# Read the Excel file "Retail_Categories.xlsx" into a pandas DataFrame
df_Retail_Categories = pd.read_excel("Retail_Categories.xlsx")

# Read the Excel file "Retail_Stores.xlsx" into a pandas DataFrame
df_Retail_Stores = pd.read_excel("Retail_Stores.xlsx")

# Display the column names of the DataFrame
df_Retail_Stores.columns

# Read the Excel file "Retail_Sales.xlsx" into a pandas DataFrame
df_Retail_Sales = pd.read_excel("Retail_Sales.xlsx")

# Display the first few rows of the DataFrame
df_Retail_Sales.head()

# Read the Excel file "Retail_Products.xlsx" into a pandas DataFrame
df_Retail_Products = pd.read_excel("Retail_Products.xlsx")

# Display the column names of the DataFrame
df_Retail_Products.columns

# Read the Excel file "Retail_Customers.xlsx" into a pandas DataFrame
df_Retail_Customers = pd.read_excel("Retail_Customers.xlsx")

# Display information about the DataFrame (e.g., column names, data types)
df_Retail_Customers.info()

# Staging

### Insert Data into Staging Tables

In [ ]:
# Define an SQL INSERT statement for inserting data into the staging.retail_stores table
insert_Retail_Stores = """INSERT INTO staging.Retail_Stores(StoreID, StoreName, Address,City, State, ZipCode,ManagerID) VALUES(?,?,?,?,?,?,?)"""

# Iterate through rows in the DataFrame and execute the INSERT statement for each row
for _, row in df_Retail_Stores.iterrows():
    cursor.execute(insert_Retail_Stores, row['StoreID'], row['StoreName'],row['Address'],row['City'],row['State'],row['ZipCode'], row['ManagerID'])

# Commit the changes to the database
cnxn.commit()

In [ ]:
# Define an SQL INSERT statement for inserting data into the staging.Retail_Brands table
insert_Retail_Brands = """INSERT INTO staging.Retail_Brands VALUES(?,?,?)"""

# Iterate through rows in the DataFrame and execute the INSERT statement for each row
for _, row in df_Retail_Brands.iterrows():
    cursor.execute(insert_Retail_Brands, row['BrandID'], row['BrandName'],row['Description'])

# Commit the changes to the database
cnxn.commit()

In [ ]:
# Similar INSERT statements are repeated for other tables like staging.Retail_Sales, staging.Retail_Products, staging.Retail_Categories, staging.Retail_Employees, staging.Retail_Customers, etc.

insert_Retail_Sales = """INSERT INTO staging.Retail_Sales VALUES(?,?,?,?,?,?,?,?)"""
for _, row in df_Retail_Sales.iterrows():
    cursor.execute(insert_Retail_Sales, row['TransactionID'], row['CustomerID'],row['ProductID'],row['Quantity'],row['UnitPrice'],row['TotalAmount'],row['TransactionDate'],row['StoreID'])

cnxn.commit()

In [ ]:
insert_Retail_Products = """INSERT INTO staging.Retail_Products VALUES(?,?,?,?,?,?,?)"""
for _, row in df_Retail_Products.iterrows():
    cursor.execute(insert_Retail_Products, row['ProductID'], row['ProductName'],row['CategoryID'],row['BrandID'], row['Description'],row['UnitPrice'],row['UnitsInStock'])

cnxn.commit()

In [ ]:
insert_Retail_Categories = """INSERT INTO staging.Retail_Categories VALUES(?,?,?)"""
 
for _, row in df_Retail_Categories.iterrows():
    cursor.execute(insert_Retail_Categories, row['CategoryID'], row['CategoryName'],row['Description'])

cnxn.commit()

In [ ]:
insert_Retail_Employees = """INSERT INTO staging.Retail_Employees VALUES(?,?,?,?,?,?,?)"""
for _, row in df_Retail_Employees.iterrows():
    cursor.execute(insert_Retail_Employees, row['EmployeeID'], row['FirstName'],row['LastName'],row['Position'], row['HireDate'],row['ContactNumber'],row['Email'])

cnxn.commit()

In [ ]:
insert_Retail_Customers = """INSERT INTO staging.Retail_Customers VALUES(?,?,?,?,?,?,?,?,?,?,?)"""
for _, row in df_Retail_Customers.iterrows():
    cursor.execute(insert_Retail_Customers, row['CustomerID'], row['FirstName'],row['LastName'],row['Gender'], row['DateOfBirth'],row['Address'],row['City'],row['State'], row['ZipCode'],row['Phone'],row['Email'])

cnxn.commit()

# ODS (Operational Data Store) 

### Data Cleaning

In [ ]:
# Clean phone numbers in the df_Retail_Customers DataFrame by removing non-digits and filtering for exactly 10 digits
df_Retail_Customers['Phone'] = df_Retail_Customers['Phone'].str.replace(r'\D', '', regex=True)
df_Retail_Customers['Phone'] = df_Retail_Customers['Phone'].apply(lambda x: x if len(x) == 10 else '0')

# Display the cleaned DataFrame
print(df_Retail_Customers)

In [ ]:
# Clean contact numbers in the df_Retail_Employees DataFrame using similar steps as above
insert_Retail_Employees = """INSERT INTO ods.Retail_Employees VALUES(?,?,?,?,?,?,?)"""
for _, row in df_Retail_Employees.iterrows():
    if(row['ContactNumber'] == '0'):
        cursor.execute(insert_Retail_Employees, row['EmployeeID'],row['FirstName'],row['LastName'],row['Position'], None ,row['HireDate'],row['Email'])
    else:
        cursor.execute(insert_Retail_Employees, row['EmployeeID'], row['FirstName'],row['LastName'],row['Position'],row['ContactNumber'] ,row['HireDate'],row['Email'])
        
cnxn.commit()

### Insert Data into ODS Tables

In [ ]:
# Insert data into ods.Retail_Customers table
insert_Retail_Customers = """INSERT INTO ods.Retail_Customers VALUES(?,?,?,?,?,?,?,?,?,?,?)"""
for _, row in df_Retail_Customers.iterrows():
    cursor.execute(insert_Retail_Customers, row['CustomerID'], row['FirstName'],row['LastName'],row['Gender'], row['DateOfBirth'],row['Address'],row['City'],row['State'], row['ZipCode'],row['Phone'],row['Email'])
cnxn.commit()

In [ ]:
# Insert data into ods.Retail_Brands table
insert_Retail_Brands = """INSERT INTO ods.Retail_Brands VALUES(?,?,?)"""
for _, row in df_Retail_Brands.iterrows():
    cursor.execute(insert_Retail_Brands, row['BrandID'], row['BrandName'],row['Description'])
cnxn.commit()

In [ ]:
# Insert data into ods.Retail_Categories table
insert_Retail_Categories = """INSERT INTO ods.Retail_Categories VALUES(?,?,?)"""
for _, row in df_Retail_Categories.iterrows():
    cursor.execute(insert_Retail_Categories, row['CategoryID'], row['CategoryName'],row['Description'])
cnxn.commit()

In [ ]:
# Insert data into ods.Retail_Stores table
insert_Retail_Stores = """INSERT INTO ods.Retail_Stores VALUES(?,?,?,?,?,?,?)"""
for _, row in df_Retail_Stores.iterrows():
    cursor.execute(insert_Retail_Stores, row['StoreID'], row['StoreName'],row['Address'],row['City'],row['State'],row['ZipCode'], row['ManagerID'])
cnxn.commit()

In [ ]:
# Insert data into ods.Retail_Products table
insert_Retail_Products = """INSERT INTO ods.Retail_Products VALUES(?,?,?,?,?,?,?)"""
for _, row in df_Retail_Products.iterrows():
    cursor.execute(insert_Retail_Products, row['ProductID'], row['ProductName'],row['CategoryID'],row['BrandID'], row['Description'],row['UnitPrice'],row['UnitsInStock'])
cnxn.commit()

In [ ]:
# Insert data into ods.Retail_Sales table
insert_Retail_Sales = """INSERT INTO ods.Retail_Sales VALUES(?,?,?,?,?,?,?,?)"""
for _, row in df_Retail_Sales.iterrows():
    cursor.execute(insert_Retail_Sales, row['TransactionID'], row['CustomerID'],row['ProductID'],row['Quantity'],row['UnitPrice'],row['TotalAmount'],row['TransactionDate'], row['StoreID'])
cnxn.commit()

# DWH (Data Warehouse) 

### Insert Data into DWH Tables

In [ ]:
# Insert data into dwh.SalesFact table (Fact Table)
insert_Sales_Fact = """INSERT INTO dwh.SalesFact VALUES(?,?,?,?,?,?,?,?)"""
for _, row in df_Retail_Sales.iterrows():
    cursor.execute(insert_Sales_Fact, row['TransactionID'], row['CustomerID'],row['ProductID'], row['StoreID'],row['Quantity'],row['UnitPrice'], (row['UnitPrice']*row['Quantity']),row['TransactionDate'])

cnxn.commit()

In [ ]:
# Insert data into dwh.EmployeesDim table
insert_Emp_dim = """INSERT INTO dwh.EmployeesDim VALUES(?,?,?,?,?,?,?)"""
for _, row in df_Retail_Employees.iterrows():
    # Check if ContactNumber is '0' (indicating a missing or invalid value)
    if (row['ContactNumber'] == '0'):
        # If ContactNumber is '0', insert NULL for ContactNumber in the database
        cursor.execute(insert_Emp_dim, row['EmployeeID'], row['FirstName'], row['LastName'], row['Position'], row['HireDate'], None, row['Email'])
    else:
        # Otherwise, insert the ContactNumber value
        cursor.execute(insert_Emp_dim, row['EmployeeID'], row['FirstName'], row['LastName'], row['Position'], row['HireDate'], row['ContactNumber'], row['Email'])

cnxn.commit()

In [ ]:
# Insert data into dwh.CustomersDim table
insert_Customers_Dim = """INSERT INTO dwh.CustomersDim VALUES(?,?,?,?,?,?,?,?,?,?,?)"""
for _, row in df_Retail_Customers.iterrows():
    # Check if Phone number is '0' (indicating a missing or invalid value)
    if (row['Phone'] == '0'):
        # If Phone number is '0', insert NULL for Phone in the database
        cursor.execute(insert_Customers_Dim, row['CustomerID'], row['FirstName'], row['LastName'], row['Gender'], row['DateOfBirth'], row['Address'], row['City'], row['State'], row['ZipCode'], None, row['Email'])
    else:
        # Otherwise, insert the Phone number value
        cursor.execute(insert_Customers_Dim, row['CustomerID'], row['FirstName'], row['LastName'], row['Gender'], row['DateOfBirth'], row['Address'], row['City'], row['State'], row['ZipCode'], row['Phone'], row['Email'])

cnxn.commit()

In [ ]:
# Insert data into dwh.StoresDim table
insert_Stores_Dim = """INSERT INTO dwh.StoresDim VALUES(?,?,?,?,?,?,?)"""
for _, row in df_Retail_Stores.iterrows():
    cursor.execute(insert_Stores_Dim, row['StoreID'], row['StoreName'], row['Address'], row['City'], row['State'], row['ZipCode'], row['ManagerID'])

cnxn.commit()

In [ ]:
# Insert data into dwh.SalesFact table
insert_Sales_Fact = """INSERT INTO dwh.SalesFact VALUES(?,?,?,?,?,?,?,?)"""
for _, row in df_Retail_Sales.iterrows():
    cursor.execute(insert_Sales_Fact, row['TransactionID'], row['CustomerID'], row['ProductID'], row['StoreID'], row['Quantity'], row['UnitPrice'], (row['UnitPrice'] * row['Quantity']), row['TransactionDate'])

cnxn.commit()

In [ ]:
# Insert data into dwh.BrandsDim table
insert_Retail_Brands = """INSERT INTO dwh.BrandsDim VALUES(?,?)"""
for _, row in df_Retail_Brands.iterrows():
    cursor.execute(insert_Retail_Brands, row['BrandID'], row['BrandName'])
    
cnxn.commit()

In [ ]:
# Insert data into dwh.CategoriesDim table
insert_CategoriesDim = """INSERT INTO dwh.CategoriesDim VALUES (?, ?)"""
for _, row in df_Retail_Categories.iterrows():
    cursor.execute(insert_CategoriesDim, row['CategoryID'], row['CategoryName'])

cnxn.commit()

In [ ]:
# Insert data into dwh.ProductsDim table
insert_ProductsDim = """INSERT INTO dwh.ProductsDim VALUES(?,?,?,?,?,?)"""
for _, row in df_Retail_Products.iterrows():
    cursor.execute(insert_ProductsDim, row['ProductID'], row['ProductName'], row['CategoryID'], row['BrandID'], row['UnitPrice'], row['UnitsInStock'])

cnxn.commit()

# Close Database Connection

In [ ]:
# Close the connection to the SQL Server
cnxn.close()